<a href="https://colab.research.google.com/github/Ochodus/DBLab_onehot/blob/main/Subsetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random
from google.colab.data_table import DataTable

In [3]:
origin_data = pd.read_excel('drive/MyDrive/DBLab_Onehot/mental_health_medical_data.xlsx')
DataTable.max_columns = 200

In [4]:
#증상을 array형태로 변환
def tokenizer(df):
  tokenized = df.copy()
  del_token = "[']"
  all_token = []

  for k, s in enumerate(df['symptoms']):
    symptoms = []
    for t in del_token:
      s = s.replace(t, "")
    s = s.split(',')
    for i in s:
      if len(i) > 0 and i[0] == " ":
        i = i[1:]
      symptoms.append(i)
    if symptoms[0] == "":
      symptoms.clear()

    all_token.append(symptoms)

  tokenized.drop("symptoms", axis=1, inplace=True)
  tokenized.insert(2, "symptoms", all_token, True)

  return tokenized

#각 질병의 증상 집합에서 모든 부분집합 구하기
def subset(df):
  subsetted = df.copy()
  subset = []
  for i, s in enumerate(df['symptoms']):
    if len(s) == 0:
      continue
    subsets = [[]]
    max_len = pow(2, len(s)-1)
    for t in s:
      size = len(subsets)
      for k in range(size):
        if size == max_len:
          continue
        subsets.append(subsets[k]+[t])
    for subs in subsets:
      target = subsetted['disease'].ne(df['disease'][i]).idxmin()
      new_row = pd.DataFrame([[df['disease'][i], subs, '정신건강의학과']], columns = subsetted.columns)
      subsetted = pd.concat([subsetted.iloc[:target+1], new_row, subsetted.iloc[target+1:]], ignore_index = True)

  return subsetted

In [5]:
origin_data['symptoms']
tokenized = tokenizer(origin_data)
tokenized.drop('Unnamed: 0', axis=1, inplace=True)
tokenized

,disease,symptoms,department
0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,공황 장애,"[숨막히는 느낌, 얼굴이 화끈거림, 가슴 통증, 죽음에 대한 공포, 어지러움, 발한...",정신건강의학과
3,금단 현상,"[복시, 손떨림, 고혈압, 서맥, 저혈압, 청색증, 불안, 기운없음, 불면증, 두통...",정신건강의학과
4,분열정동 장애,"[와해된 언어, 피해망상, 환각, 환시, 환청, 와해된 행동, 망상, 이상행동]",정신건강의학과
...,...,...,...
59,혈관성 치매,"[언어장애, 인지장애, 감각 이상, 마비]",정신건강의학과
60,히스테리성 인격장애,[],정신건강의학과
61,레트 증후군,"[상동적 행동, 인지장애, 운동장애, 정신 지체]",정신건강의학과
62,폐소공포증,"[불안, 좁은 공간에서의 불안감]",정신건강의학과


In [ ]:
subsetted = subset(origin_data)
subsetted.to_excel('drive/MyDrive/DBLab_Onehot/subset_data.xlsx')